In [1]:
import pandas as pd
import boto3
import json

In [2]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.ini'))

KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')

CLUSTER_TYPE = config.get("CLUSTER", "CLUSTER_TYPE")
NUM_NODES = config.get("CLUSTER", "NUM_NODES")
NODE_TYPE = config.get("CLUSTER", "NODE_TYPE")

CLUSTER_IDENTIFIER = config.get("CLUSTER", "CLUSTER_IDENTIFIER")
DB = config.get("CLUSTER", "DB")
DB_USER = config.get("CLUSTER", "DB_USER")
DB_PASSWORD = config.get("CLUSTER", "DB_PASSWORD")
PORT = config.get("CLUSTER", "PORT")

IAM_ROLE_NAME = config.get("IAM_ROLE", "ROLE_NAME")

(DB_USER, DB_PASSWORD, DB)

('beshoy', 'Subiyah241050', 'sparkify')

In [3]:
pd.DataFrame({"Param":
              ["CLUSTER_TYPE", "NUM_NODES", "NODE_TYPE", "CLUSTER_IDENTIFIER",
                  "DB", "DB_USER", "DB_PASSWORD", "PORT", "IAM_ROLE_NAME"],
              "Value":
                  [CLUSTER_TYPE, NUM_NODES, NODE_TYPE, CLUSTER_IDENTIFIER,
                      DB, DB_USER, DB_PASSWORD, PORT, IAM_ROLE_NAME]
              })

,Param,Value
0,CLUSTER_TYPE,multi-node
1,NUM_NODES,4
2,NODE_TYPE,dc2.large
3,CLUSTER_IDENTIFIER,SparkifyCluster
4,DB,sparkify
5,DB_USER,beshoy
6,DB_PASSWORD,Subiyah241050
7,PORT,5439
8,IAM_ROLE_NAME,myRedshiftRole


In [4]:
import boto3


iam = boto3.client('iam', aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET,
                   region_name='us-east-1'
                   )

redshift = boto3.client('redshift',
                        region_name="us-east-1",
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET
                        )

In [5]:
# from botocore.exceptions import ClientError

# #1.1 Create the role,
# try:
#     print("1.1 Creating a new IAM Role")
#     dwhRole = iam.create_role(
#         Path='/',
#         RoleName=IAM_ROLE_NAME,
#         Description = "Allows Redshift clusters to call AWS services on your behalf.",
#         AssumeRolePolicyDocument=json.dumps(
#             {'Statement': [{'Action': 'sts:AssumeRole',
#                'Effect': 'Allow',
#                'Principal': {'Service': 'redshift.amazonaws.com'}}],
#              'Version': '2012-10-17'})
#     )
# except Exception as e:
#     print(e)


# print("1.2 Attaching Policy")

# iam.attach_role_policy(RoleName=IAM_ROLE_NAME,
#                        PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
#                       )['ResponseMetadata']['HTTPStatusCode']


print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.3 Get the IAM role ARN
arn:aws:iam::339713167706:role/myRedshiftRole


In [13]:
redshift = boto3.client('redshift',
                        region_name="us-east-1",
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET
                        )

try:
    response = redshift.create_cluster(
        # HW
        ClusterType=CLUSTER_TYPE,
        NodeType=NODE_TYPE,
        NumberOfNodes=int(NUM_NODES),

        # Identifiers & Credentials
        DBName=DB,
        ClusterIdentifier=CLUSTER_IDENTIFIER,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,

        # Roles (for s3 access)
        IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

In [15]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', None)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus",
                  "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k, v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])


myClusterProps = redshift.describe_clusters(
    ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,sparkifycluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,beshoy
4,DBName,sparkify
5,Endpoint,"{'Address': 'sparkifycluster.ctqlbky7lyjr.us-east-1.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-0f69dde8e6e5cee0e
7,NumberOfNodes,4


In [9]:
ENDPOINT = myClusterProps["Endpoint"]["Address"]
ROLE_ARN = myClusterProps["IamRoles"][0]["IamRoleArn"]
print("ENDPOINT :: ", ENDPOINT)
print("ROLE_ARN :: ", ROLE_ARN)

ENDPOINT ::  sparkifycluster.ctqlbky7lyjr.us-east-1.redshift.amazonaws.com
ROLE_ARN ::  arn:aws:iam::339713167706:role/myRedshiftRole


In [17]:
ec2 = boto3.resource('ec2',
                     region_name="us-east-1",
                     aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET
                     )

In [18]:
try:
    vpc = ec2.Vpc(id=myClusterProps["VpcId"])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp="0.0.0.0/0",
        IpProtocol="TCP",
        FromPort=int(PORT),
        ToPort=int(PORT),
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-085f2c968098463e9')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


{'Cluster': {'ClusterIdentifier': 'sparkifycluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'beshoy',
  'DBName': 'sparkify',
  'Endpoint': {'Address': 'sparkifycluster.ctqlbky7lyjr.us-east-1.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2024, 1, 7, 17, 30, 24, 434000, tzinfo=tzutc()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-085f2c968098463e9',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-0f69dde8e6e5cee0e',
  'AvailabilityZone': 'us-east-1a',
  'PreferredMaintenanceWindow': 'thu:08:30-thu:09:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfN